In [36]:
!unzip drive-download-20241220T223313Z-001.zip -d pdfs


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Archive:  drive-download-20241220T223313Z-001.zip
  inflating: pdfs/Дополнительная_методичка_по_Амазону.pdf  
  inflating: pdfs/Руководство_по_бортовому_компьютеу_Аматрон.pdf  
  inflating: pdfs/Руководство по эксплуатации ISO BUS.pdf  
  inflating: pdfs/AI-помощник инженера.docx  
  inflating: pdfs/Копия Копия 5_Презентация_ПрофАгро.pptx  
  inflating: pdfs/Кртакое_руководство_по_разбрасователю_Амазон_ZG_TS_7501.pdf  
  inflating: pdfs/Руководство_по_разбрасователю_Амазон_ZG_TS_7501.pdf  


In [1]:
import os

print(os.name)  # 'posix' for Linux/Mac, 'nt' for Windows


posix


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0


In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [3]:
!pip install pymupdf            # For 'fitz' (PyMuPDF)
!pip install pillow             # For 'PIL' (Image)
!pip install langchain          # For 'langchain.text_splitter'
!pip install transformers       # For 'transformers.AutoTokenizer' and 'transformers.AutoModelForCausalLM'
!pip install torch              # For 'torch'         # For 'faiss' (CPU version; use 'faiss-gpu' for GPU sup!port)
!pip install numpy   

In [4]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 37.5 MB/s eta 0:00:00:00:0100:01


In [34]:
import fitz  # PyMuPDF
from PIL import Image
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import faiss
import numpy as np
import os
import glob
import io

# Путь к папке с PDF файлами
pdf_folder_path = "pdfs"


In [35]:
# Функция для обработки PDF файлов
def process_pdfs_from_folder(pdf_folder_path):
    all_text, all_images = [], []
    pdf_files = glob.glob(os.path.join(pdf_folder_path, "*.pdf"))

    for pdf_path in pdf_files:
        print(f"Обрабатываем файл: {pdf_path}")
        text, images = [], []

        # Открытие и извлечение данных из PDF
        with fitz.open(pdf_path) as pdf:
            for page in pdf:
                text.append(page.get_text())  # Извлекаем текст

                # Извлекаем и сохраняем изображения
                for img_index, img in enumerate(page.get_images(full=True)):
                    xref = img[0]
                    base_image = pdf.extract_image(xref)
                    image_bytes = base_image["image"]
                    image = Image.open(io.BytesIO(image_bytes))
                    image_bytes_io = io.BytesIO()
                    image.save(image_bytes_io, format=base_image["ext"].upper())
                    images.append(image_bytes_io.getvalue())

        all_text.append(" ".join(text))
        all_images.extend(images)

    return all_text, all_images, pdf_files


In [36]:
# Разбиение текста на чанки
def split_text_into_chunks(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
    return splitter.split_text(text)


In [33]:
!pip install --upgrade transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

In [25]:
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import numpy as np
import faiss
import re

In [26]:

model_st = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Чистка текста
def clean_text(text):
    text = re.sub(r"[^а-яА-Яa-zA-Z0-9\s.,!?]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def preprocess_chunks(chunks):
    cleaned_chunks = [clean_text(chunk) for chunk in chunks]
    filtered_chunks = [chunk for chunk in cleaned_chunks if len(chunk.split()) > 5]
    return filtered_chunks

# Сохранение и загрузка Faiss-индекса
def save_embeddings_to_faiss(embeddings):
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    faiss.write_index(index, "faiss_index.index")
    return index

def load_faiss_index(index_path):
    return faiss.read_index(index_path)

def get_embeddings_st(texts):
    return model_st.encode(texts, show_progress_bar=True)

def rank_with_bm25(chunks, query):
    tokenized_chunks = [chunk.split() for chunk in chunks]
    bm25 = BM25Okapi(tokenized_chunks)
    query_tokens = query.split()
    scores = bm25.get_scores(query_tokens)
    ranked_chunks = sorted(
        zip(chunks, scores), key=lambda x: x[1], reverse=True
    )
    return ranked_chunks

def search_similar_chunks_with_images(query, all_chunks, chunk_image_map, index, embeddings, k=5):
    """
    Searches for similar text chunks using Faiss, ranks them with BM25, and retrieves corresponding images.
    """
    # Perform Faiss search
    query_embedding = get_embeddings_st([query])[0]
    D, I = index.search(np.array([query_embedding]), k=10)  # Retrieve top-10 results from Faiss

    # Get text chunks from Faiss results
    similar_chunks = [all_chunks[idx] for idx in I[0]]

    # Rank the chunks using BM25
    ranked_chunks = rank_with_bm25(similar_chunks, query)

    # Retrieve top-k chunks and their corresponding images
    similar_chunks_with_images = []
    for chunk, _ in ranked_chunks[:k]:
        idx = all_chunks.index(chunk)  # Find the original index of the chunk
        image = chunk_image_map.get(idx)  # Get the associated image
        similar_chunks_with_images.append((chunk, image))

    return similar_chunks_with_images


In [38]:
def save_images(images):
    os.makedirs("images", exist_ok=True)
    image_paths = []
    for idx, image in enumerate(images):
        image_path = os.path.join("images", f"image_{idx}.png")
        with open(image_path, "wb") as f:
            f.write(image)
        image_paths.append(image_path)
    return image_paths

In [29]:
def process_pdfs(pdf_folder_path):
    all_text, all_images, pdf_files = process_pdfs_from_folder(pdf_folder_path)
    print('1done')
    # Разбиение текста на чанки
    all_chunks = []
    for text in all_text:
        all_chunks.extend(split_text_into_chunks(text))
    print('2done')
    all_chunks = preprocess_chunks(all_chunks)
    # Генерация эмбеддингов и сохранение их в Faiss
    embeddings = get_embeddings_st(all_chunks)
    print('2.1done')
    index = save_embeddings_to_faiss(np.array(embeddings))
    print('3done')
    # Сохранение изображений
    image_paths = save_images(all_images)
    print('4done')
    # Создание отображения чанков и изображений
    chunk_image_map = {i: image_paths[i] for i in range(len(all_images))}
    print('5done')
    # Статистика
    print(f"Обработано {len(pdf_files)} PDF файлов.")
    print(f"Чанков текста: {len(all_chunks)}.")
    print(f"Изображений: {len(all_images)}.")
    print("Эмбеддинги сохранены в Faiss, изображения в папке 'images'.")

    return all_chunks, chunk_image_map, index, embeddings


In [59]:
!rm -r images

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [39]:
pdf_folder_path = "pdfs"  # Укажите путь к папке с PDF файлами
all_chunks, chunk_image_map, index, embeddings = process_pdfs(pdf_folder_path)

Обрабатываем файл: pdfs/Руководство_по_бортовому_компьютеу_Аматрон.pdf
Обрабатываем файл: pdfs/Руководство по эксплуатации ISO BUS.pdf
Обрабатываем файл: pdfs/Кртакое_руководство_по_разбрасователю_Амазон_ZG_TS_7501.pdf
Обрабатываем файл: pdfs/Руководство_по_разбрасователю_Амазон_ZG_TS_7501.pdf
1done
2done


Batches:   0%|          | 0/27 [00:00<?, ?it/s]

2.1done
3done
4done
5done
Обработано 4 PDF файлов.
Чанков текста: 861.
Изображений: 2268.
Эмбеддинги сохранены в Faiss, изображения в папке 'images'.


In [82]:
query = "калибровка значений с помощью весов"
similar_chunks_with_images = search_similar_chunks_with_images(query, all_chunks, chunk_image_map, index, embeddings, k=5)

print("Similar chunks with images:")
for i, (chunk, image) in enumerate(similar_chunks_with_images, start=1):
    print(f"Chunk {i}: {chunk}")
    print(f"Image: {image}")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similar chunks with images:
Chunk 1: отверстие. Коэффициент калибровки 13 определять Открыть заслонку Дождаться наполнения сборной емкости Врем 0 с отмена 5. Взвесьте собранное удобрение с учетом массы приемной мкости. 6. Введите результат взвешивания с учетом единицы измерения. Калибровка разбрасывателя 26 ввести собранное количество 5.00 кг Отобразится новый калибровочный коэффициент. 7. Сохраните новый коэффициент калибровки, прервите калибровку, Повторите калибровку с заново рассчитанным коэффициентом. Калибровка разбрасывателя 33 новый коэффициент калибровки 1.00 повторить отмена сохранить Вводопределениеуправление данными для конкретного удобрения ISOBUS TS BAG0204.8 01.24 23 7.4.2 Определение коэффициента калибровки с помощью заслонки для мелкого особого разбрасываемого материала Подготовка к калибровке
Image: images/image_253.png
Chunk 2: Настройка агрегата ISOBUS TS BAG0204.8 01.24 41 9.3 Разбрасыватель со взвешиванием тарировка разбрасывателя удобрений Тарировка служит для оп

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="MTSAIR/Cotype-Nano", device="cuda")

messages = [

  {"role": "system", "content": "Ты — ИИ-помощник фермера тракториста. Тебе дано задание: необходимо сгенерировать емкий но полный ответ, отвечающий на поставленный вопрос, используя в качестве базы знаний поступающие тебе тексты."},

  {"role": "user", "content": "Расскажи мне про ИИ"},

]

res = pipe(messages, max_length=1024)

print(res[0]['generated_text'][-1]['content'])

In [68]:
pipe2 = pipeline("text-generation", model="MTSAIR/Cotype-Nano", device="cuda")
messages2 = [
  {"role": "system", "content": "Ты — ИИ-помощник по правильности речи. Проверяй входящие фразы и исправляй грамматичексие, семантические и лексические ошибки, следи за структурой речи. Выведи только исправленную фразу (если она была неправильной, иначе - оставь как есть)"},
  {"role": "user", "content": "росскажи мене про искуственый интелект"},
]
res2 = pipe2(messages2, max_length=1024)
print(res2[0]['generated_text'][-1]['content'])

Device set to use cuda


Расскажи мне про искусственный интеллект.


In [86]:
!pip install 'accelerate>=0.26.0'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [90]:
import torch

# Clear all allocated tensors to free memory
torch.cuda.empty_cache()

# Optionally force garbage collection (not always necessary)
import gc
gc.collect()


9863

In [1]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the current GPU device
    device = torch.cuda.current_device()
    print(f"Device: {torch.cuda.get_device_name(device)}")
    
    # Get memory details
    total_memory = torch.cuda.get_device_properties(device).total_memory
    allocated_memory = torch.cuda.memory_allocated(device)
    cached_memory = torch.cuda.memory_reserved(device)
    free_memory = total_memory - allocated_memory - cached_memory
    
    print(f"Total Memory: {total_memory / 1024**3:.2f} GB")
    print(f"Allocated Memory: {allocated_memory / 1024**3:.2f} GB")
    print(f"Cached Memory: {cached_memory / 1024**3:.2f} GB")
    print(f"Free Memory: {free_memory / 1024**3:.2f} GB")
else:
    print("CUDA is not available.")


Device: NVIDIA A100-PCIE-40GB
Total Memory: 39.38 GB
Allocated Memory: 0.00 GB
Cached Memory: 0.00 GB
Free Memory: 39.38 GB


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
torch.manual_seed(42)

model_name = "t-tech/T-lite-it-1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype="auto",
)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")



2024-12-21 11:10:22.057408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734779422.073166    3616 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734779422.077481    3616 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 11:10:22.090532: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
def grammar(prompt):

    messages = [
        {"role": "system", "content": "Ты — ИИ-помощник по правильности речи. Проверяй входящие фразы и исправляй грамматичексие, семантические и лексические ошибки, следи за структурой речи. Выведи только исправленную фразу (если она была неправильной, иначе - оставь как есть)"},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=256
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response


In [41]:
def helper_agent(similar_chunks, query):
    similar_chunks_text = "\n\n".join(similar_chunks)
    messages = [
        {"role": "system", "content": "Ты — ИИ-помощник фермера тракториста. Тебе дано задание: необходимо сгенерировать емкий но полный ответ, отвечающий на поставленный вопрос, используя в качестве базы знаний поступающие тебе тексты."},
        {"role": "user", "content": f"Вопрос: {query}\n\nТексты для анализа:\n{similar_chunks_text}"}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=4096
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response


In [45]:
def get_ans(question):
    # Preprocess the query
    query = grammar(question)
    
    # Search for similar chunks and associated images
    similar_chunks_with_images = search_similar_chunks_with_images(
        query=query,
        all_chunks=all_chunks,
        chunk_image_map=chunk_image_map,
        index=index,
        embeddings=embeddings,
        k=5  # Retrieve top-5 results
    )
    
    # Extract chunks (without images) for further processing
    similar_chunks = [chunk for chunk, _ in similar_chunks_with_images]
    similar_chunks_text = "\n\n".join(similar_chunks)
    
    # Pass similar chunks and query to helper_agent for generating response
    res = helper_agent(similar_chunks, query)

    # Debug the structure of res
    print(f"helper_agent output: {res}")
    
    # Since `res` is a string, directly return it
    return similar_chunks_with_images, res


In [18]:
import pandas as pd

In [19]:
df = pd.read_csv("test.csv")

In [46]:
similar_chunks_list = []
answers_list = []

# Iterate over each row (assuming the question is in a column named 'question')
for question in df['user_input']:
    similar_chunks, answer = get_ans(question)
    similar_chunks_list.append(similar_chunks)  # Append the list directly
    answers_list.append(answer)

# Add results to new columns in the DataFrame
df2['contexts'] = similar_chunks_list  # Store the list directly
df2['response'] = answers_list

# Save the updated DataFrame back to a new CSV file
df2.to_csv("submission.csv", index=False)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

helper_agent output: Чтобы настроить норму внесения удобрений на распределителе ZG-TS, следуйте следующим шагам:

1. **Откройте панель управления распределителя**: Это позволит вам получить доступ к интерфейсу для настройки параметров.

2. **Найдите параметр "Норма внесения" или аналогичный**: Обычно он находится в разделе настроек или управления удобрениями.

3. **Установите требуемую норму внесения удобрений**: Следуйте инструкциям производителя, которые могут быть представлены в виде таблицы норм внесения для конкретного типа удобрений. Учитывайте такие факторы, как диаметр семян и удельный вес, так как они могут влиять на количество вносимого удобрения.

4. **Подтвердите изменения и проверьте корректность установок**: После внесения изменений убедитесь, что все параметры настроены правильно. Для этого можно использовать функцию калибровки, например, метод оффлайн калибровки или автоматическую калибровку удобрения.

Для более точной настройки рекомендуется использовать руководство п

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

helper_agent output: Перед первым вводом в эксплуатацию устройства ZG-TS необходимо выполнить ряд обязательных шагов, чтобы обеспечить безопасную и эффективную работу. Вот подробная инструкция:

1. **Ознакомьтесь с инструкцией пользователя**: Перед началом эксплуатации устройства ZG-TS обязательно ознакомьтесь с руководством пользователя. Это поможет вам понять все особенности и функции устройства, а также соблюдать требования по технике безопасности. Убедитесь, что все пользователи, которые будут работать с устройством, также ознакомились с руководством.

2. **Проверьте комплектность оборудования**: Убедитесь, что все компоненты устройства присутствуют и находятся в рабочем состоянии. Это включает в себя проверку всех кабелей, адаптеров и других аксессуаров, входящих в комплект поставки.

3. **Подключите все необходимые кабели и адаптеры**: Следуйте инструкциям в руководстве по подключению всех кабелей и адаптеров. Это может включать подключение GPS-приемника, если он предусмотрен кон

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

helper_agent output: Для настройки параметров загрузки данных в приложении myAmaRouter, чтобы они соответствовали вашему заданию, выполните следующие шаги:

1. **Откройте приложение myAmaRouter** и перейдите в раздел настроек или параметров загрузки.
   
2. **Установите необходимые параметры**:
   - **Формат файла**: выберите подходящий формат, который поддерживает приложение (например, ISOXML).
   - **Частота обновления**: установите частоту, которая наиболее подходит для вашего задания (например, каждые 10 минут или по мере необходимости).
   - **Другие опции**: настройте дополнительные параметры, такие как тип данных, которые будут загружены, и любые специфические требования вашего задания.

3. **Сохраните изменения**, если это необходимо, и подтвердите их на AmaTron 4, следуя инструкциям в руководстве пользователя.

Если у вас возникают трудности с выбором конкретных параметров, обратитесь к руководству пользователя или поддержке приложения. Например, если требуется импорт данных и

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

helper_agent output: Чтобы предотвратить перекрытия при распределении удобрений, важно использовать современные технологии и функции, которые обеспечивают точность и эффективность процесса. Вот ключевые моменты, которые следует учесть:

1. **Система GPS-навигации**: Используйте GPS для точного определения местоположения и планирования маршрута распределения. Это поможет избежать повторного внесения удобрений в одни и те же участки.

2. **Автоматическая коррекция траектории**: Включите функцию автоматической коррекции, чтобы система могла адаптироваться к изменениям в рельефе или других факторах, влияющих на траекторию распределения.

3. **Функция контроля скорости**: Поддерживайте постоянную скорость движения, чтобы обеспечить равномерное распределение удобрений. Это особенно важно для предотвращения перекрытий.

4. **Система управления расходом**: Настройте систему управления расходом для точного дозирования удобрений. Это позволит избежать как недостаточного, так и избыточного внесен

NameError: name 'df2' is not defined

In [48]:
df['contexts'] = similar_chunks_list  # Store the list directly
df['response'] = answers_list

In [49]:
print(df.columns)

Index(['user_input', 'contexts', 'response'], dtype='object')


In [50]:
df = df.drop('user_input', axis=1)

In [51]:
df.to_csv("submission.csv", index=False)